In [126]:
import pickle
from pymania import *
import matplotlib.pyplot as plt
import pickle as pk
import numpy as np
from scipy.spatial.distance import squareform,pdist
from collections import defaultdict
import networkx as nx

In [127]:
r_rois = [f'R{i}' for i in range(1,181)]
r_roi2ind = {}
r_ind2roi = {}
for i,roi in enumerate(sorted(r_rois)):
    r_roi2ind[roi]=i
    r_ind2roi[i]=roi

def link_type(x,q1,q2):
    if x<q1:
        return 0#'short-range'
    elif x>q2:
        return 2#'long-range'
    else:
        return 1#'mid-range'

with open('R_ROIs.txt') as f:
    st = f.readlines()
    r_roi_names = [xx.strip().split('_')[1] for xx in st]

l_rois = [f'L{i}' for i in range(1,181)]
l_roi2ind = {}
l_ind2roi = {}
for i,roi in enumerate(sorted(l_rois)):
    l_roi2ind[roi]=i
    l_ind2roi[i]=roi

with open('L_ROIs.txt') as f:
    st = f.readlines()
    l_roi_names = [xx.strip().split('_')[1] for xx in st]

In [128]:
def net2G(net,hem='l'):
    G = nx.DiGraph()
    rois = l_rois if hem=='l' else r_rois;
    ind2roi = l_ind2roi if hem=='l' else r_ind2roi;
    for roi in rois:
        G.add_node(roi)
    for i in range(180):
        for j in range(180):
            if i==j or net[i,j]<.5:continue
            r1 = ind2roi[i]
            r2 = ind2roi[j]
            G.add_edge(r1,r2)
    return G

In [194]:
with open('sparse_right_37_group.pk','rb') as f:
    RG = pk.load(f)
with open('sparse_left_37_group.pk','rb') as f:
    LG = pk.load(f)

In [195]:
R = net2G(RG[0],'r')

In [196]:
L = net2G(LG[0])

In [199]:
zr = sorted([(xx,R.degree(xx)) for xx in r_rois],reverse=True,key = lambda x:x[-1])[0:5]
zrs = set([xx[0] for xx in zr])

In [200]:
zl = sorted([(xx,L.degree(xx)) for xx in l_rois],reverse=True,key = lambda x:x[-1])[0:5]
zls = set([xx[0] for xx in zl])

In [201]:
#right and left rich club
print("'LEFT'","''","''","'RIGHT'","''","''")
for i in range(5):
    nl = zl[i][0]
    i1 = int(nl[1:])-1
    r1 = l_roi_names[i1]
    nr = zr[i][0]
    i2 = int(nr[1:])-1
    r2 = r_roi_names[i2]
    print(nl,r1,'X',nr,r2,'X')

'LEFT' '' '' 'RIGHT' '' ''
L106 PoI2 X R106 PoI2 X
L167 PoI1 X R131 TGd X
L131 TGd X R143 PGp X
L151 PGs X R167 PoI1 X
L149 PFm X R148 PF X


In [161]:
l_rc=0
l_fd=0
l_lc=0
for v,w in L.edges():
    if (v in zls) and (w in zls):
        l_rc+=1
    elif (v in zls) or (w in zls):
        l_fd+=1
    else:
        l_lc+=1

In [165]:
with open('axonal.pk','rb') as f:
    A = pk.load(f)

In [186]:
r_rc=[]
r_fd=[]
r_lc=[]
E = A['l']
lengths = [E[i,j] for i in range(179) for j in range(i+1,180)]
q1,q2 = np.percentile(lengths,[25,75])
for v,w in L.edges():
    i = int(v[1:])-1
    j = int(w[1:])-1
    d = E[i,j]
    if (v in zls) and (w in zls):
        r_rc+=[link_type(d,q1,q2)]
    elif (v in zls) or (w in zls):
        r_fd+=[link_type(d,q1,q2)]
    else:
        r_lc+=[link_type(d,q1,q2)]

In [179]:
from collections import Counter

In [187]:
Counter(r_rc)

Counter({0: 60, 1: 73, 2: 10})

In [188]:
Counter(r_lc)

Counter({0: 2971, 1: 879, 2: 176})

In [189]:
Counter(r_fd)

Counter({0: 844, 1: 811, 2: 152})

In [185]:
966/(3059+966+201)

0.2285849503076195